# <center>Emotion Prediction</center>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.utils import class_weight

In [ ]:
labels_path = "emotion_dataset/labels"

label_counts = Counter()

for label_file in os.listdir(labels_path):
    if label_file.endswith(".txt"):
        with open(os.path.join(labels_path, label_file), "r") as f:
            for line in f:
                parts = line.strip().split()
                if parts:  
                    label = int(parts[0])   
                    label_counts[label] += 1

labels = sorted(label_counts.keys())
counts = [label_counts[l] for l in labels]

plt.figure(figsize=(8,5))
sns.barplot(x=labels, y=counts, palette="pastel")
plt.title("Number of Pictures per Label Category")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

To improve accuracy we will remove all off the labes except 4,5 and 6. Which equal to Happy, Neutral and Sad.

In [ ]:
path_images = "emotion_dataset/images"
path_labels = "emotion_dataset/labels"

images = []
labels = []

for img_file in os.listdir(path_images):
    img_id, _ = os.path.splitext(img_file)
    label_file = os.path.join(path_labels, img_id + ".txt")
    
    if not os.path.exists(label_file):
        continue
    
    with open(label_file, "r") as f:
        parts = f.readline().strip().split()
        if not parts:
            continue
        label = int(parts[0])
    
    if label not in [4, 5, 6]:
        continue
    
    with Image.open(os.path.join(path_images, img_file)) as img:
        images.append(img.copy())
        labels.append(label)

df = pd.DataFrame({"Images": images, "Labels": labels})

print("Loaded:", len(df), "images from categories 4, 5, 6")

counts = df["Labels"].value_counts().sort_index()

# Plot
plt.figure(figsize=(8,5))
sns.barplot(x=counts.index, y=counts.values, palette="pastel")
plt.title("Number of Pictures per Label Category")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

print(df["Labels"].value_counts())



In [ ]:
display(df["Images"][3])
print(df["Labels"][3])

In [ ]:
display(df["Images"][4])
print(df["Labels"][4])

In [ ]:
display(df["Images"][5000])
print(df["Labels"][5000])

In [ ]:
x = []
for i in range(len(df)):
    img_resized = df["Images"].iloc[i].resize((96, 96), Image.Resampling.LANCZOS)
    df.at[i, "Images"] = img_resized
    x.append(np.asarray(img_resized))

x = np.array(x) / 255.0 
y = np.array(df["Labels"])

In [ ]:
y_mapped = y - 4 
num_classes = 3
y_onehot = to_categorical(y_mapped, num_classes=num_classes)

x_train, x_test, y_train, y_test = train_test_split(
    x, y_onehot, test_size=0.2, stratify=y_mapped, random_state=42
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./1.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./1.)

train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
test_generator = test_datagen.flow(x_test, y_test, batch_size=32)


In [ ]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, (3,3), activation="relu", input_shape=(96, 96, 3)))
emotion_model.add(MaxPooling2D((2,2)))
emotion_model.add(Conv2D(64, (3,3), activation="relu"))
emotion_model.add(MaxPooling2D((2,2)))
emotion_model.add(Conv2D(128, (3,3), activation="relu"))
emotion_model.add(MaxPooling2D((2,2)))
emotion_model.add(Flatten())
emotion_model.add(Dense(64, activation="relu"))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(num_classes, activation="softmax"))

emotion_model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"]
)

In [ ]:
y_labels = y_mapped
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_labels),
    y=y_labels
)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

history = emotion_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    shuffle=True,
    class_weight=class_weights_dict
)

test_loss, test_accuracy = emotion_model.evaluate(test_generator, verbose=1)
print("Final Test Loss (Emotion):", test_loss)
print("Final Test Accuracy (Emotion):", test_accuracy)

emotion_model.save("emotion_model.h5")

In [ ]:
def process_and_predict_emotion(file):
    
    im = Image.open(file)
    width, height = im.size

  
    if width != height:
        if width > height:
            left = width / 2 - height / 2
            right = width / 2 + height / 2
            top = 0
            bottom = height
        else:
            left = 0
            right = width
            top = 0
            bottom = width
        im = im.crop((left, top, right, bottom))

    im = im.resize((96, 96), Image.Resampling.LANCZOS)

    ar = np.asarray(im).astype("float32") / 255.0
    ar = ar.reshape(-1, 96, 96, 3) 

    pred_probs = emotion_model.predict(ar)[0]          
    pred_class = np.argmax(pred_probs)                 
    emotion_label = pred_class + 4                     

    prediction = {"emotion": emotion_label}
    print(f"{file} -> Predicted Emotion: {emotion_label}")

    return im, prediction

In [ ]:
test_folder = "test"
results = {}

for filename in os.listdir(test_folder):
    if not (filename.lower().endswith(".jpg") or filename.lower().endswith(".png")):
        continue  
    
    img_path = os.path.join(test_folder, filename)
    im_resized, prediction = process_and_predict_emotion(img_path)
    
    
    results[filename] = prediction
    
    
    plt.imshow(im_resized)
    plt.axis('off')
    plt.title(f"{filename} → Emotion: {prediction['emotion']}")
    plt.show()

In [ ]:
model = load_model("emotion_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("emotion_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="emotion_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
def process_and_predict_emotion_tflite(file):

    im = Image.open(file).convert("RGB")
    
    im_resized = im.resize((96, 96), Image.Resampling.LANCZOS)

    ar = np.asarray(im_resized).astype('float32') / 255.0
    ar = np.expand_dims(ar, axis=0)  

    interpreter.set_tensor(input_details[0]['index'], ar)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    emotion_classes = ["Happy", "Neutral", "Sad"]
    predicted_idx = np.argmax(output_data)
    predicted_emotion = emotion_classes[predicted_idx]

    return im_resized, {"emotion": predicted_emotion, "raw_output": output_data}

In [ ]:
test_folder = "test"
results = {}

for filename in os.listdir(test_folder):
    if not (filename.lower().endswith(".jpg") or filename.lower().endswith(".png")):
        continue 
    
    img_path = os.path.join(test_folder, filename)
    im_resized, prediction = process_and_predict_emotion_tflite(img_path)
    
    results[filename] = prediction
    
    plt.imshow(im_resized)
    plt.axis("off")
    plt.title(f"{filename} → Emotion: {prediction["emotion"]}")
    plt.show()

In [ ]:
def predict_tflite(model_path, x_data, task="binary"):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    preds = []

    for i in range(len(x_data)):
        input_data = x_data[i].reshape(1, 96, 96, 3).astype(input_details[0]["dtype"])
        interpreter.set_tensor(input_details[0]["index"], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]["index"])
        preds.append(output_data[0])

    preds = np.array(preds)

    if task == "multi_class":
        preds = np.argmax(preds, axis=1) 
    elif task == "binary":
        preds = np.round(preds).astype(int).reshape(-1)
    else:
        raise ValueError("task must be 'binary' or 'multi_class'")

    return preds



def evaluate_model(model_path, x_test, y_test, model_name="", task="binary"):
    preds = predict_tflite(model_path, x_test, task=task)

    if len(y_test.shape) > 1 and y_test.shape[1] > 1:
        y_test = np.argmax(y_test, axis=1)

    precision = precision_score(y_test, preds, average="weighted", zero_division=0)
    recall = recall_score(y_test, preds, average="weighted", zero_division=0)
    f1 = f1_score(y_test, preds, average="weighted", zero_division=0)
    accuracy = accuracy_score(y_test, preds)

    print(f" Results for {model_name} ({model_path}):")
    print(f"  Accuracy : {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall   : {recall:.4f}")
    print(f"  F1-Score : {f1:.4f}")
    print("-" * 40)


evaluate_model("emotion_model.tflite", x_test, y_test, model_name="Emotion Model", task="multi_class")